In [203]:
#writer = pd.ExcelWriter('output.xlsx')
#occ_data.to_excel(writer,'Sheet1')
#writer.save()


import warnings
warnings.filterwarnings('ignore')

#import required Libraries
import pandas as pd
import os
import math
import numpy as np

#read in the data files
files = os.listdir("/Users/student/Desktop/Capstone/Data")

#change working directory to where the excel files are
os.chdir('C:\\Users\\student\\Desktop\\Capstone\\Data')

In [204]:
#check to make sure read in worked
for i in range(len(files)):
    print (files[i])

AsianF.xls
AsianM.xls
BlackF.xls
BlackM.xls
census_final.xlsx
HawaiianF.xls
HawaiianM.xls
HispanicF.xls
HispanicM.xls
IndianF.xls
IndianM.xls
other
OtherF.xls
OtherM.xls
output.xlsx
TotalF.xls
TotalM.xls
TwoRaceF.xls
TwoRaceM.xls
WhiteF.xls
WhiteM.xls


In [205]:
#create a list of the dataset names
dataset_names=[]

for i in range(len(files)):
    dataset_names.append(files[i][:files[i].find(".")])


In [206]:
#create a dictionary with the dataset name and the actual data file
cenus_data={}

#range(len(dataset_names))
for i in range(len(dataset_names)):
    if(files[i][-3:]=="xls"):
      cenus_data[dataset_names[i]]=pd.read_excel(files[i])

    #at this point have all the occupation data sets in a dictionary with the name and then the file

In [207]:
#make sure dictionary worked
cenus_data.keys()

#pull the actual data set in the dictionary
#cenus_data[list(cenus_data.keys())[2]]

dict_keys(['AsianF', 'AsianM', 'BlackF', 'BlackM', 'HawaiianF', 'HawaiianM', 'HispanicF', 'HispanicM', 'IndianF', 'IndianM', 'OtherF', 'OtherM', 'TotalF', 'TotalM', 'TwoRaceF', 'TwoRaceM', 'WhiteF', 'WhiteM'])

In [208]:
#reassign the column names as the name of the dataset PLUS the current column name

#list of the columns that we want
female_columns=['NAME','PCT086130', 'PCT086143','PCT086181','PCT086216','PCT086219']
male_columns=['NAME','PCT086004', 'PCT086017','PCT086055','PCT086090','PCT086093']

for x in range(len(cenus_data)):
    new_col_names=[]
    
    #check to see if it is a male or female data set
    #create a temp table called final that contains only the variables in the female/male columns list along with name
    if(list(cenus_data.keys())[x][-1:]=='F'):
        for i in range(0,len(female_columns)):
            if(i==0):
                final=cenus_data[list(cenus_data.keys())[x]].filter(regex=female_columns[i])
            else:
                temp2 = cenus_data[list(cenus_data.keys())[x]].filter(regex=female_columns[i])
                final=pd.concat([final,temp2],axis=1)
    else:
        for i in range(0,len(male_columns)):
            if(i==0):
                final=cenus_data[list(cenus_data.keys())[x]].filter(regex=male_columns[i])
            else:
                temp2 = cenus_data[list(cenus_data.keys())[x]].filter(regex=male_columns[i])
                final=pd.concat([final,temp2],axis=1)
                
    #assign the temp data set to the corresponding spot in the census data list            
    cenus_data[list(cenus_data.keys())[x]]=final

    #keep the first two columns the same name: county number and name
    #for all the other columns add the data set name infront of it
    for i in range(len(final.columns)):
        if(i<1):
            new_col_names.append(final.columns[i])
        else:
            new_col_names.append(list(cenus_data.keys())[x]+"_"+final.columns[i])
    #end the loop changing column names
        
    #assign the newly created column name list to the dataset
    cenus_data[list(cenus_data.keys())[x]].columns=new_col_names

#end loop going through each dataset

In [209]:
#combine all the datasets in the cenus_data dictionary into one

occ_data=cenus_data[list(cenus_data.keys())[len(cenus_data)-1]]
    
for x in range(0,len(cenus_data)-1):
    occ_data=pd.merge(occ_data,cenus_data[list(cenus_data.keys())[x]],how="left",on="NAME")
    
#remove the duplicate county variable    
#occ_data=occ_data.drop(["COUNTY_x","COUNTY_y"],axis=1)

In [210]:
#start reading in and cleaning up the income statistics dataset

In [211]:
income=pd.read_excel("/Users/student/Desktop/Capstone/Data/other/income_statistics.xls")



In [212]:
#remove first two rows
income=income.loc[2:]

#if a value in the first row is blank change it to the first value to the left
for i in range(len(income.columns)):
    if(type(income.iat[0,i])!=str and math.isnan(income.iat[0,i])):
        income.iat[0,i]=income.iat[0,i-1]
        


In [213]:
#combine the first and second row and make that the column headers
#loop through the number of columns
col_names=[]
for i in range(len(income.columns)):
    #if the second row is blank then add just the first row
    if(type(income.iat[1,i])!=str and math.isnan(income.iat[1,i])):
        col_names.append(income.iat[0,i])
    #if the second row isnot blank then combine the first and second row
    else:
        col_names.append(income.iat[0,i]+"_"+income.iat[1,i])
        
#assign the new column names
income.columns=col_names

#delete the first two rows which are now the column names
income=income.drop(income.index[:3])

In [214]:
#delete blanks, 99999 and 0 in zip code column

#delete if first column is blank
income = income[pd.notnull(income.iloc[:,0])]
#income = income[pd.notnull(income.iloc[:,1])]

#delete zip codes of 0
income=income[income.iloc[:,0] != 0]

#delete zip codes of 99999
income=income[income.iloc[:,0] != 99999]

#delete rows with a missing number of returns. This gets rid of non zip codes in first column
income = income[pd.notnull(income.iloc[:,2])]


In [215]:
#create the number of returns in each income bracket columns

one_25=income[income.iloc[:,1] == '$1 under $25,000'].iloc[:,:3]
one_25=one_25.drop("Size of adjusted gross income",axis=1)
one_25=one_25.rename(columns = {'Number of returns':'$1 under $25,000 Number of Returns'})

twentyfive_50=income[income.iloc[:,1] == '$25,000 under $50,000'].iloc[:,:3]
twentyfive_50=twentyfive_50.drop("Size of adjusted gross income",axis=1)
twentyfive_50=twentyfive_50.rename(columns = {'Number of returns':'$25,000 under $50,000 Number of Returns'})

fifty_75=income[income.iloc[:,1] == '$50,000 under $75,000'].iloc[:,:3]
fifty_75=fifty_75.drop("Size of adjusted gross income",axis=1)
fifty_75=fifty_75.rename(columns = {'Number of returns':'$50,000 under $75,000 Number of Returns'})

seventyfive_100=income[income.iloc[:,1] == '$75,000 under $100,000'].iloc[:,:3]
seventyfive_100=seventyfive_100.drop("Size of adjusted gross income",axis=1)
seventyfive_100=seventyfive_100.rename(columns = {'Number of returns':'$75,000 under $100,000 Number of Returns'})

hundo_200=income[income.iloc[:,1] == '$100,000 under $200,000'].iloc[:,:3]
hundo_200=hundo_200.drop("Size of adjusted gross income",axis=1)
hundo_200=hundo_200.rename(columns = {'Number of returns':'$100,000 under $200,000 Number of Returns'})

twohundo_up=income[income.iloc[:,1] == '$200,000 or more'].iloc[:,:3]
twohundo_up=twohundo_up.drop("Size of adjusted gross income",axis=1)
twohundo_up=twohundo_up.rename(columns = {'Number of returns':'$200,000 or more Number of Returns'})


#merge these with the income dataset to create them in columns
income=pd.merge(income,one_25,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,twentyfive_50,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,fifty_75,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,seventyfive_100,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,hundo_200,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")

income=pd.merge(income,twohundo_up,left_on="ZIP\ncode [1]", right_on="ZIP\ncode [1]", how="left")



In [216]:
#delete everything besides the total row
income = income[-pd.notnull(income.iloc[:,1])]

#remove the gross income column
income=income.drop("Size of adjusted gross income", axis=1)


In [217]:
#conver all columns to ints
income=income.astype(int)

In [218]:
#match the county with the zip code

In [219]:
#load in the zip codes file
zip_codes=pd.read_excel("/Users/student/Desktop/Capstone/Data/other/zip_codes.xlsx")

In [220]:
#merge the income and zip code file together
income=pd.merge(income,zip_codes,left_on=income.iloc[:,0],right_on=zip_codes.iloc[:,0])
income=income.drop(["ZIP\ncode [1]", 'ZIP Code'],axis=1)

In [221]:
#summarize the variables by county

income=income.groupby("County",as_index=False).sum()


In [222]:
#merge the occupation and income datasets

In [223]:
#remove county from the name in occupation
for i in range(len(occ_data)):
    occ_data["NAME"][i]=occ_data["NAME"][i].replace(" County","")

In [224]:
#actually merge the occupation and income datasets

census=pd.merge(occ_data,income, left_on="NAME",right_on="County")



In [225]:
census=census.drop(['County'],axis=1)

In [226]:
writer = pd.ExcelWriter('output.xlsx')
census.to_excel(writer,'Sheet1')
writer.save()